In [1]:
from tqdm import tqdm
import pickle

In [52]:
import sys
import os

# Đường dẫn đến thư mục mini_model
mini_model_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Lấy danh sách tất cả thư mục con trong mini_model
for folder in os.listdir(mini_model_path):
    folder_path = os.path.join(mini_model_path, folder)
    # Chỉ thêm nếu là thư mục và không phải file
    if os.path.isdir(folder_path):
        if folder_path not in sys.path:
            sys.path.append(folder_path)

# Import module
import review_quality
import json

In [2]:
with open('../../dataset\\info_hotel\\feature_url.json', 'r', encoding='utf-8') as f:
    feature_url = json.load(f)

In [7]:
ids = [key['id'] for key in feature_url[0:1000]]

In [8]:
hotel_review_stats = review_quality.HotelReviewStats()
hotel_review_stats.load_stats()
score_review_quality = hotel_review_stats.calculate_final_score_local_data(ids, print_warning=False)

Stats loaded from D:\graduate_dissertation\final\dataset\review_hotel\review_quality.json


In [1]:
import os
import json

folder_path = "D:\\graduate_dissertation\\final\\dataset\\review_hotel\\absa_result"
all_data = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_data.extend(data)


In [2]:
print(len(all_data))

897892


In [15]:
data = all_data

In [16]:
len(all_data)

897892

In [20]:
all_data[202]

{'id_review': '571b57a0-d3d2-451a-b80a-5b420e802cf0',
 'review_positive': 'Phòng khá tôt',
 'translated_review_positive': 'The room is quite good',
 'review_negative': 'Phòng sạch sẽ',
 'translated_review_negative': 'Clean room',
 'result': [['room', 'Positive'],
  ['comfort', 'Positive'],
  ['cleanliness', 'Positive']],
 'id': '10007210',
 'review_title': 'Rất tốt',
 'review_date': 'Ngày đánh giá: ngày 13 tháng 7 năm 2023',
 'review_score': 'Đạt điểm 8,0',
 'review_photo': None}

In [21]:
from tqdm import tqdm 
clean_data =[]
for i in tqdm(range(0, len(all_data)), total=len(all_data)):
    try:
        clean_data.append({
            "id_hotel": all_data[i]['id'],
            "id_review": all_data[i]['id_review'],
            "country": all_data[i].get('country', None),
            "room": all_data[i].get('room', None),
            "state": all_data[i].get('state', None),
            "rating": all_data[i]['review_score'],
            "criteria": all_data[i]['result'],
            "date": all_data[i]['date']
        })
    except:
        clean_data.append({
            "id_hotel": all_data[i]['id'],
            "id_review": all_data[i]['id_review'],
            "country": all_data[i].get('country', None),
            "room": all_data[i].get('room', None),
            "state": all_data[i].get('state', None),
            "rating": all_data[i]['review_score'],
            "criteria": all_data[i]['result'],
            "date": all_data[i]['review_date']
        })

100%|██████████| 897892/897892 [00:01<00:00, 780050.92it/s]


In [22]:
len(clean_data)

897892

In [23]:
clean_data[0]

{'id_hotel': '10000593',
 'id_review': '2367e2c2-8bb2-4a79-bc48-034fde283649',
 'country': 'Việt Nam',
 'room': 'Bungalow Deluxe Nhìn Ra Khu Vườn',
 'state': 'Nhóm',
 'rating': 'Đạt điểm 10',
 'criteria': [['room', 'Positive'],
  ['facilities', 'Positive'],
  ['view', 'Positive'],
  ['environment', 'Positive'],
  ['comfort', 'Positive'],
  ['food', 'Positive'],
  ['service', 'Positive']],
 'date': '1 đêm · tháng 9/2023'}

In [24]:
# Mapping từ 16 tiêu chí ban đầu sang 9 tiêu chí mới
criteria_mapping = {
    "location": "Vị trí & môi trường",
    "room": "Phòng nghỉ & tiện nghi",
    "cleanliness": "Sạch sẽ",
    "comfort": "Phòng nghỉ & tiện nghi",
    "service": "Dịch vụ & nhân viên",
    "staff": "Dịch vụ & nhân viên",
    "wifi": "Wi-Fi",
    "food": "Ẩm thực",
    "bathroom": "Phòng nghỉ & tiện nghi",
    "parking": "Cơ sở vật chất",
    "value": "Giá trị",
    "facilities": "Cơ sở vật chất",
    "air conditioning": "Phòng nghỉ & tiện nghi",
    "view": "Vị trí & môi trường",
    "environment": "Vị trí & môi trường",
    "security": "An ninh"
}


criteria_mapping_6 = {
    "Vị trí & môi trường": "Vị trí & môi trường",
    "Phòng nghỉ & tiện nghi": "Chất lượng phòng & tiện nghi",
    "Sạch sẽ": "Chất lượng phòng & tiện nghi",
    "Dịch vụ & nhân viên": "Dịch vụ & nhân viên",
    "Wi-Fi": "Chất lượng phòng & tiện nghi",
    "Ẩm thực": "Ẩm thực",
    "Cơ sở vật chất": "Cơ sở vật chất & An ninh",
    "Giá trị": "Giá trị",
    "An ninh": "Cơ sở vật chất & An ninh",
}

# Hàm chuyển đổi tiêu chí
def convert_criteria(original_criteria):
    new_criteria_dict = {}
    for old_key, sentiment in original_criteria:
        new_key = criteria_mapping.get(old_key)
        if new_key:
            # Nếu đã có tiêu chí này, bỏ qua nếu trùng sentiment
            if new_key not in new_criteria_dict:
                new_criteria_dict[new_key] = sentiment
    return [[k, v] for k, v in new_criteria_dict.items()]

def convert_criteria_6(original_criteria):
    new_criteria_dict = {}
    for old_key, sentiment in original_criteria:
        new_key = criteria_mapping_6.get(old_key)
        if new_key:
            # Nếu đã có tiêu chí này, bỏ qua nếu trùng sentiment
            if new_key not in new_criteria_dict:
                new_criteria_dict[new_key] = sentiment
    return [[k, v] for k, v in new_criteria_dict.items()]

# Áp dụng cho từng review
for review in clean_data:
    review["criteria"] = convert_criteria(review["criteria"])




In [25]:
clean_data[0:2]

[{'id_hotel': '10000593',
  'id_review': '2367e2c2-8bb2-4a79-bc48-034fde283649',
  'country': 'Việt Nam',
  'room': 'Bungalow Deluxe Nhìn Ra Khu Vườn',
  'state': 'Nhóm',
  'rating': 'Đạt điểm 10',
  'criteria': [['Phòng nghỉ & tiện nghi', 'Positive'],
   ['Cơ sở vật chất', 'Positive'],
   ['Vị trí & môi trường', 'Positive'],
   ['Ẩm thực', 'Positive'],
   ['Dịch vụ & nhân viên', 'Positive']],
  'date': '1 đêm · tháng 9/2023'},
 {'id_hotel': '10000593',
  'id_review': '590605ee-34ba-4223-b132-22e25c298289',
  'country': 'Việt Nam',
  'room': 'Bungalow Nhìn ra vườn',
  'state': 'Phòng gia đình',
  'rating': 'Đạt điểm 10',
  'criteria': [['Vị trí & môi trường', 'Positive'],
   ['Ẩm thực', 'Mixed'],
   ['Phòng nghỉ & tiện nghi', 'Positive'],
   ['Cơ sở vật chất', 'Positive'],
   ['Sạch sẽ', 'Positive']],
  'date': '2 đêm · tháng 9/2023'}]

In [26]:
with open('hotel_review_score.json', 'w', encoding='utf-8') as f:
    json.dump(clean_data, f, ensure_ascii=False, indent=4)

In [19]:
import pandas as pd
import json


# Hàm xử lý criteria để tạo summary
def process_criteria(criteria_list):
    criteria_summary = {}
    for criteria in criteria_list:
        for criterion, sentiment in criteria:
            if criterion not in criteria_summary:
                criteria_summary[criterion] = {'Positive': 0, 'Neutral': 0, 'Negative': 0, 'Mixed': 0}
            criteria_summary[criterion][sentiment] += 1
    return criteria_summary

# Hàm xử lý từng nhóm
def process_group(group):
    # Danh sách các item trong nhóm
    items = []
    all_criteria = []
    
    for _, row in group.iterrows():
        item = {
            'id_review': row['id_review'],
            'country': row['country'],
            'room': row['room'],
            'state': row['state'],
            'rating': row['rating'],
            'criteria': row['criteria']
        }
        items.append(item)
        all_criteria.append(row['criteria'])
    
    # Tạo criteria_summary cho nhóm
    criteria_summary = process_criteria(all_criteria)
    
    return {
        'items': items,
        'criteria_summary': criteria_summary
    }


# Hàm xử lý criteria để tạo summary
def process_criteria(criteria_list):
    criteria_summary = {}
    for criteria in criteria_list:
        for criterion, sentiment in criteria:
            if criterion not in criteria_summary:
                criteria_summary[criterion] = {'Positive': 0, 'Neutral': 0, 'Negative': 0, 'Mixed': 0}
            criteria_summary[criterion][sentiment] += 1
    return criteria_summary

# Hàm xử lý một nhóm
def process_group(group):
    items = []
    all_criteria = []
    
    for _, row in group.iterrows():
        item = {
            #'id_hotel': row['id_hotel'],
            'id_review': row['id_review'],
            'country': row['country'],
            'room': row['room'],
            'state': row['state'],
            'rating': row['rating'],
            'criteria': row['criteria']
        }
        items.append(item)
        all_criteria.append(row['criteria'])
    
    criteria_summary = process_criteria(all_criteria)
    
    return {
        'items': items,
        'criteria_summary': criteria_summary
    }




In [20]:
df = pd.DataFrame(clean_data)

# Gom nhóm theo id_hotel
grouped = df.groupby('id_hotel')

# Tạo dictionary kết quả
result_groupby_id = {id_hotel: process_group(group) for id_hotel, group in grouped}

In [22]:
result_groupby_id['10001679']

{'items': [{'id_review': 'd85c28aa-7928-4168-a3be-e5eee272c27f',
   'country': 'Việt Nam',
   'room': 'Phòng Giường Đôi Có Phòng Tắm Riêng',
   'state': 'Khách lẻ',
   'rating': 'Đạt điểm 10',
   'criteria': [['Cơ sở vật chất', 'Positive'],
    ['Vị trí & môi trường', 'Positive'],
    ['Phòng nghỉ & tiện nghi', 'Positive'],
    ['Dịch vụ & nhân viên', 'Positive']]},
  {'id_review': 'a66e5aaa-0333-4971-a8b8-d4da35c192ce',
   'country': 'Úc',
   'room': 'Phòng Giường Đôi Có Phòng Tắm Riêng',
   'state': 'Cặp đôi',
   'rating': 'Đạt điểm 8,0',
   'criteria': [['Phòng nghỉ & tiện nghi', 'Negative'],
    ['Dịch vụ & nhân viên', 'Positive'],
    ['Vị trí & môi trường', 'Positive'],
    ['Cơ sở vật chất', 'Positive'],
    ['Ẩm thực', 'Positive']]},
  {'id_review': '79edc3ec-1486-457d-855d-1670634d76a7',
   'country': 'Cộng hoà Séc',
   'room': 'Phòng Giường Đôi Có Phòng Tắm Riêng',
   'state': 'Khách lẻ',
   'rating': 'Đạt điểm 9,0',
   'criteria': [['Phòng nghỉ & tiện nghi', 'Positiv

In [15]:
# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(clean_data)

# Gom nhóm theo id_hotel
grouped = df.groupby('id_hotel')

# Tạo dictionary kết quả
result_groupby_id = {id_hotel: process_group(group) for id_hotel, group in grouped}

for key in tqdm(result_groupby_id.keys(), total=len(result_groupby_id.keys()), desc="Processing groups"):
    # Tạo dictionary kết quả
    result = {
        'country': {},
        'room': {},
        'state': {}
    }

    df = pd.DataFrame(result_groupby_id[key]['items']) 
    # Gom nhóm theo country
    grouped_by_country = df.groupby('country')
    for country, group in grouped_by_country:
        result['country'][country] = process_group(group)

    # Gom nhóm theo room
    grouped_by_room = df.groupby('room')
    for room, group in grouped_by_room:
        result['room'][room] = process_group(group)

    # Gom nhóm theo state
    grouped_by_state = df.groupby('state')
    for state, group in grouped_by_state:
        result['state'][state] = process_group(group)

    result_groupby_id[key]['grouped'] = result

NameError: name 'tqdm' is not defined

In [23]:
import json
from collections import defaultdict

# Hàm chuyển rating dạng chuỗi thành số
def parse_rating(rating_str):
    try:
        # Xử lý trường hợp "Đạt điểm X"
        if "Đạt điểm" in rating_str:
            number_str = rating_str.split("Đạt điểm ")[1].strip()
            # Thay dấu phẩy thành dấu chấm nếu có
            number_str = number_str.replace(",", ".")
            return float(number_str)
        
        # Xử lý trường hợp số thập phân với dấu phẩy (ví dụ: "9,0")
        rating_str = rating_str.replace(",", ".")
        return float(rating_str)
    
    except (ValueError, IndexError) as e:
        # Ghi log lỗi nếu cần (tùy chọn)
        print(f"Không thể chuyển đổi rating '{rating_str}' thành số: {e}")
        return 0  # Giá trị mặc định nếu không thể chuyển đổi


# Hàm tính tỷ lệ Positive, Neutral, Negative
def calculate_ratios(criteria_summary):
    ratios = {}
    for criterion, sentiments in criteria_summary.items():
        total = sum(sentiments.values())
        if total > 0:
            ratios[criterion] = {
                "Positive": sentiments["Positive"] / total,
                "Neutral": sentiments["Neutral"] / total,
                "Negative": sentiments["Negative"] / total,
                "Mixed": sentiments["Mixed"] / total,
                "total": total
            }
        else:
            ratios[criterion] = {"Positive": 0, "Neutral": 0, "Negative": 0, "Mixed": 0}
    return ratios

# Hàm tạo thống kê tổng quan theo khách sạn
def generate_hotel_summary(data):
    summary = {}
    
    for id_hotel, hotel_data in data.items():
        items = hotel_data["items"]
        criteria_summary = hotel_data["criteria_summary"]
        
        # Đếm số lượng review và tính rating trung bình
        review_count = len(items)
        # Lọc các item có rating hợp lệ (không None, không rỗng)
        valid_ratings = [
            parse_rating(item["rating"]) 
            for item in items 
            if item.get("rating") is not None and item.get("rating").strip() != ""
        ]
        
        avg_rating = sum(valid_ratings) / review_count if review_count > 0 else 0
        
        # Tính tỷ lệ Positive, Neutral, Negative
        criteria_ratios = calculate_ratios(criteria_summary)
        
        # Lưu kết quả vào dictionary
        summary[id_hotel] = {
            "review_count": review_count,
            "avg_rating": avg_rating,
            "criteria_summary": criteria_ratios
        }

    
    return summary



In [24]:
for key in tqdm(result_groupby_id.keys(), total=len(result_groupby_id.keys()), desc="Processing groups"):
    id_hotel = key
    criteria_summary = result_groupby_id[key]['criteria_summary']
    items = result_groupby_id[key]['items']
    review_count = len(items)
    valid_ratings = [
        parse_rating(item["rating"]) 
        for item in items 
        if item.get("rating") is not None and item.get("rating").strip() != ""
    ]
    avg_rating = sum(valid_ratings) / review_count if review_count > 0 else 0
    criteria_ratios = calculate_ratios(criteria_summary)
    result_groupby_id[key]['summary'] = {
        "review_count": review_count,
        "avg_rating": avg_rating,
        "criteria_summary": criteria_ratios
    }

Processing groups: 100%|██████████| 10570/10570 [00:00<00:00, 18608.73it/s]


In [28]:
for id in result_groupby_id.keys():
    result_groupby_id[id]['items'] = None
    result_groupby_id[id]['criteria_summary'] = None

In [32]:
with open("D:\\graduate_dissertation\\final\\mini_model\\recommenr_review_data\\review_processing_9_criteria.json", "w", encoding="utf-8") as f:
    json.dump(result_groupby_id, f, ensure_ascii=False, indent=4)

In [13]:

# Loop through each hotel ID
for hotel_id, data in result_groupby_id.items():
    criteria_summary = data['summary']['criteria_summary']
    review_count = data['summary']['review_count']
    avg_rating = data['summary']['avg_rating']
    
    # Step 1: Extract positive rates and total reviews
    positive_rates = {}
    total_reviews = {}
    for criterion, ratings in criteria_summary.items():
        positive_rates[criterion] = ratings['Positive']
        total_reviews[criterion] = ratings['total']
    
    # Step 2: Calculate weights based on review frequency
    
    weights = {criterion: count / review_count for criterion, count in total_reviews.items()}
    
    # Step 3: Compute raw score
    raw_score = 0
    for criterion in criteria_summary:
        raw_score += weights[criterion] * positive_rates[criterion]
    
    # Step 4: Normalize score
    total_weight = sum(weights.values())
    normalized_score = raw_score / total_weight if total_weight > 0 else 0
    
    # Store results
    data['summary']['raw_score'] = {
        'avg_rating': avg_rating,
        'positive_rates': positive_rates,
        'total_reviews': total_reviews,
        'weights': weights,
        'raw_score': raw_score,
        'normalized_score': normalized_score
    }

# Step 2: Calculate mu (global average positive rate)
total_positive_reviews = 0
total_reviews = 0

for hotel_id, data in result_groupby_id.items():
    review_count = data['summary']['review_count']
    criteria_summary = data['summary']['criteria_summary']
    
    # Sum positive reviews for all criteria in this hotel
    for criterion, ratings in criteria_summary.items():
        positive_reviews = ratings['Positive'] * ratings['total']
        total_positive_reviews += positive_reviews
    
        # Sum total review_count for this hotel
        total_reviews += ratings['total']

mu = total_positive_reviews / total_reviews if total_reviews > 0 else 0
print(f"Calculated mu (global average positive rate): {mu:.4f}")


# Step 3: Bayesian Averaging
C = 50  # Hyperparameter
bayesian_results = []

for key, data in result_groupby_id.items():
    n_h = data['summary']['review_count']
    normalized_score = data['summary']['raw_score']['normalized_score']
    
    # Bayesian Score: (n_h * Normalized_Score_h + C * mu) / (n_h + C)
    bayesian_score = (n_h * normalized_score + C * mu) / (n_h + C)
    
    # Store results
    data['summary']['bayesian_score'] = {
        'normalized_score': normalized_score,
        'bayesian_score': bayesian_score
    }


Calculated mu (global average positive rate): 0.8170


In [14]:
for key in tqdm(result_groupby_id.keys(), total=len(result_groupby_id.keys()), desc="Processing groups"):
    for key1 in result_groupby_id[key]['grouped']['country'].keys():
        criteria_summary = result_groupby_id[key]['grouped']['country'][key1]['criteria_summary']
        items = result_groupby_id[key]['grouped']['country'][key1]['items']
        review_count = len(items)
        valid_ratings = [
            parse_rating(item["rating"]) 
            for item in items 
            if item.get("rating") is not None and item.get("rating").strip() != ""
        ]
        avg_rating = sum(valid_ratings) / review_count if review_count > 0 else 0
        criteria_ratios = calculate_ratios(criteria_summary)
        result_groupby_id[key]['grouped']['country'][key1]['summary'] = {
            "review_count": review_count,
            "avg_rating": avg_rating,
            "criteria_summary": criteria_ratios
        }
    for key2 in result_groupby_id[key]['grouped']['room'].keys():
        criteria_summary = result_groupby_id[key]['grouped']['room'][key2]['criteria_summary']
        items = result_groupby_id[key]['grouped']['room'][key2]['items']
        review_count = len(items)
        valid_ratings = [
            parse_rating(item["rating"]) 
            for item in items 
            if item.get("rating") is not None and item.get("rating").strip() != ""
        ]
        avg_rating = sum(valid_ratings) / review_count if review_count > 0 else 0
        criteria_ratios = calculate_ratios(criteria_summary)
        result_groupby_id[key]['grouped']['room'][key2]['summary'] = {
            "review_count": review_count,
            "avg_rating": avg_rating,
            "criteria_summary": criteria_ratios
        }
    for key3 in result_groupby_id[key]['grouped']['state'].keys():
        criteria_summary = result_groupby_id[key]['grouped']['state'][key3]['criteria_summary']
        items = result_groupby_id[key]['grouped']['state'][key3]['items']
        review_count = len(items)
        valid_ratings = [
            parse_rating(item["rating"]) 
            for item in items 
            if item.get("rating") is not None and item.get("rating").strip() != ""
        ]
        avg_rating = sum(valid_ratings) / review_count if review_count > 0 else 0
        criteria_ratios = calculate_ratios(criteria_summary)
        result_groupby_id[key]['grouped']['state'][key3]['summary'] = {
            "review_count": review_count,
            "avg_rating": avg_rating,
            "criteria_summary": criteria_ratios
        }

Processing groups: 100%|██████████| 10570/10570 [00:04<00:00, 2525.03it/s]


In [15]:
import pandas as pd

# Giả sử result_groupby_id là dict chứa dữ liệu như bạn đã mô tả

rows = []

for id_, id_data in result_groupby_id.items():
    for country, country_data in id_data['grouped']['country'].items():
        criteria_summary = country_data['summary']['criteria_summary']
        row = {
            'id': id_,
            'country': country
        }
        
        for criterion, sentiment_scores in criteria_summary.items():
            positive_score = sentiment_scores.get('Positive', 0.0)
            row[f"{criterion}_Positive"] = positive_score
        
        rows.append(row)

# Tạo DataFrame với cột id, country ở đầu
df = pd.DataFrame(rows)




In [16]:
country = set()
for key in result_groupby_id.keys():
    for key1 in result_groupby_id[key]['grouped']['country'].keys():
        country.add(key1)
len(country)

247

In [17]:
country

{'',
 'Afghanistan',
 'Ai Cập',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua & Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Azerbaijan',
 'Áo',
 'Ba Lan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belize',
 'Benin',
 'Bermuda',
 'Bỉ',
 'Bolivia',
 'Bonaire St Eustatius và Saba',
 'Bosnia và Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam (Brunây)',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Butan',
 'Bồ Đào Nha',
 'Bờ Biển Ngà',
 'Bắc Macedonia',
 'Bắc Triều Tiên',
 'Cameroon',
 'Campuchia',
 'Canada',
 'Cape Verde',
 'Chad',
 'Chile',
 'Colombia',
 'Comoros',
 'Congo (Công gô)',
 'Congo (Cộng hòa Dân chủ Công gô)',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus (Cộng hoà Síp)',
 'Các tiểu đảo của Hoa Kỳ',
 'Cộng hoà Séc',
 'Cộng hòa Dominica',
 'Cộng hòa Gabon',
 'Djibouti',
 'Dominica',
 'Ecuador',
 'El Salvador',
 'Equitorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Ga

In [26]:
import numpy as np
import pandas as pd
import tensorly as tl
from tensorly.decomposition import parafac

# Thiết lập tham số
num_countries = len(country)    # Số quốc gia
num_criteria = 16      # Số tiêu chí
rank = 50              # Số latent factors cho CP Decomposition (giảm để tối ưu hiệu suất)

# Danh sách 16 tiêu chí
criteria = [
    'location_Positive', 'room_Positive', 'cleanliness_Positive', 'comfort_Positive',
    'service_Positive', 'staff_Positive', 'wifi_Positive', 'food_Positive',
    'bathroom_Positive', 'parking_Positive', 'value_Positive', 'facilities_Positive',
    'air conditioning_Positive', 'view_Positive', 'environment_Positive', 'security_Positive'
]

# Hàm tạo tensor từ DataFrame
def create_tensor_from_df(df):
    # Lấy danh sách quốc gia và khách sạn duy nhất
    countries = sorted(df['country'].unique())
    hotels = sorted(df['id'].unique())
    
    num_countries_actual = len(countries)
    num_hotels_actual = len(hotels)
    
    # Ánh xạ quốc gia và khách sạn thành chỉ số
    country_to_idx = {c: i for i, c in enumerate(countries)}
    hotel_to_idx = {h: i for i, h in enumerate(hotels)}
    
    # Tạo tensor với giá trị NaN
    tensor = np.full((num_countries_actual, num_hotels_actual, num_criteria), np.nan)
    
    # Điền dữ liệu vào tensor
    for _, row in df.iterrows():
        country_idx = country_to_idx[row['country']]
        hotel_idx = hotel_to_idx[row['id']]
        for crit_idx, crit in enumerate(criteria):
            tensor[country_idx, hotel_idx, crit_idx] = row[crit]
    
    return tensor, country_to_idx, hotel_to_idx

# Hàm tiền xử lý: Điền giá trị thiếu bằng trung bình
def preprocess_tensor(tensor):
    # Điền NaN bằng trung bình của mỗi tiêu chí
    for k in range(tensor.shape[2]):
        slice_k = tensor[:, :, k]
        mean_k = np.nanmean(slice_k)
        slice_k[np.isnan(slice_k)] = mean_k
        tensor[:, :, k] = slice_k
    return tensor

# Hàm dự đoán điểm số cho (quốc gia, khách sạn)
def predict_scores(country_idx, hotel_idx, factors, weights):
    u, v, w = factors
    scores = np.zeros(num_criteria)
    for r in range(len(weights)):
        scores += weights[r] * u[country_idx, r] * v[hotel_idx, r] * w[:, r]
    # Đảm bảo điểm số nằm trong khoảng [0, 1]
    scores = np.clip(scores, 0, 1)
    return scores


# Hàm lưu model
def save_model(weights, factors, country_idx, hotel_to_idx, filename='model.pkl'):
    model_data = {
        'weights': weights,
        'factors': factors,
        'country_idx': country_idx,
        'hotel_to_idx': hotel_to_idx,
        'criteria': criteria
    }
    with open(filename, 'wb') as f:
        pickle.dump(model_data, f)
    print(f"Model đã được lưu vào {filename}")

# Hàm tải model
def load_model(filename='model.pkl'):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} không tồn tại")
    with open(filename, 'rb') as f:
        model_data = pickle.load(f)
    return (model_data['weights'], model_data['factors'],
            model_data['country_idx'], model_data['hotel_to_idx'], model_data['criteria'])


# Hàm dự đoán sử dụng model đã lưu
def predict_with_saved_model(statucountrys, hotel, model_filename='model.pkl'):
    weights, factors, country_idx, hotel_to_idx, criteria = load_model(model_filename)
    
    # Kiểm tra trạng thái và khách sạn có trong dữ liệu không
    if country not in country_idx:
        raise ValueError(f"Trạng thái '{country}' không có trong dữ liệu")
    if hotel not in hotel_to_idx:
        raise ValueError(f"Khách sạn '{hotel}' không có trong dữ liệu")
    
    status_idx = country_idx[country]
    hotel_idx = hotel_to_idx[hotel]
    predicted_scores = predict_scores(status_idx, hotel_idx, factors, weights)
    
    # In kết quả
    print(f"\nDự đoán điểm số cho trạng thái '{country}', khách sạn '{hotel}':")
    for i, (crit, score) in enumerate(zip(criteria, predicted_scores), 1):
        print(f"{crit}: {score:.3f}")
    
    return predicted_scores


In [27]:

# Tạo tensor từ dữ liệu
print("Tạo tensor từ dữ liệu...")
tensor, country_to_idx, hotel_to_idx = create_tensor_from_df(df)

# Tiền xử lý tensor
print("Tiền xử lý tensor...")
tensor = preprocess_tensor(tensor)

# Phân rã tensor bằng CP Decomposition
print("Phân rã tensor bằng CP Decomposition...")
weights, factors = parafac(tensor, rank=rank, init='random', tol=1e-6)




Tạo tensor từ dữ liệu...
Tiền xử lý tensor...
Phân rã tensor bằng CP Decomposition...


In [30]:
print("Lưu model...")
save_model(weights, factors, country_to_idx, hotel_to_idx, filename='CF_country_for_user.pkl')

country = df['country'].iloc[1]  # ấy quốc gia mẫu
hotel = df['id'].iloc[0]         # Lấy khách sạn mẫu
country_idx = country_to_idx[country]
hotel_idx = hotel_to_idx[hotel]
predicted_scores = predict_scores(country_idx, hotel_idx, factors, weights)

# In kết quả
print(f"\nDự đoán điểm số cho trạng thái '{country}', khách sạn '{hotel}' (từ model vừa huấn luyện):")
for i, (crit, score) in enumerate(zip(criteria, predicted_scores), 1):
    print(f"{crit}: {score:.3f}")

# Dự đoán lại bằng model đã lưu để kiểm tra
print("\nKiểm tra dự đoán với model đã lưu...")
predict_with_saved_model(country, hotel, model_filename='CF_country_for_user.pkl')

Lưu model...
Model đã được lưu vào CF_country_for_user.pkl

Dự đoán điểm số cho trạng thái 'Ba Lan', khách sạn '10000593' (từ model vừa huấn luyện):
location_Positive: 0.751
room_Positive: 0.546
cleanliness_Positive: 0.727
comfort_Positive: 0.583
service_Positive: 0.757
staff_Positive: 0.900
wifi_Positive: 0.934
food_Positive: 0.675
bathroom_Positive: 0.526
parking_Positive: 0.949
value_Positive: 0.768
facilities_Positive: 0.562
air conditioning_Positive: 0.944
view_Positive: 0.810
environment_Positive: 0.725
security_Positive: 0.942

Kiểm tra dự đoán với model đã lưu...

Dự đoán điểm số cho trạng thái 'Ba Lan', khách sạn '10000593':
location_Positive: 0.751
room_Positive: 0.546
cleanliness_Positive: 0.727
comfort_Positive: 0.583
service_Positive: 0.757
staff_Positive: 0.900
wifi_Positive: 0.934
food_Positive: 0.675
bathroom_Positive: 0.526
parking_Positive: 0.949
value_Positive: 0.768
facilities_Positive: 0.562
air conditioning_Positive: 0.944
view_Positive: 0.810
environment_Positive

array([0.75055218, 0.54633263, 0.72738447, 0.58303121, 0.75732165,
       0.89997367, 0.93438911, 0.67463062, 0.52591652, 0.94919597,
       0.76834447, 0.56246264, 0.94431062, 0.81031963, 0.72461201,
       0.94204826])

In [31]:
state = set()
for key in result_groupby_id.keys():
    for key3 in result_groupby_id[key]['grouped']['state'].keys():
        state.add(key3)
len(state)

5

In [1]:
import pandas as pd

# Giả sử result_groupby_id là dict chứa dữ liệu như bạn mô tả

rows = []

for id_, id_data in result_groupby_id.items():
    for state, state_data in id_data['grouped']['state'].items():
        criteria_summary = state_data['summary']['criteria_summary']
        row = {
            'id': id_,
            'state': state
        }

        for criterion, sentiment_scores in criteria_summary.items():
            positive_score = sentiment_scores.get('Positive', 0.0)
            row[f"{criterion}_Positive"] = positive_score

        rows.append(row)

# Tạo DataFrame
df = pd.DataFrame(rows)



NameError: name 'result_groupby_id' is not defined

In [33]:
import numpy as np
import pandas as pd
import tensorly as tl
from tensorly.decomposition import parafac
import pickle
import os


statuses = ['', 'Cặp đôi', 'Khách lẻ', 'Nhóm', 'Phòng gia đình']

criteria = [
    'location_Positive', 'room_Positive', 'cleanliness_Positive', 'comfort_Positive',
    'service_Positive', 'staff_Positive', 'wifi_Positive', 'food_Positive',
    'bathroom_Positive', 'parking_Positive', 'value_Positive', 'facilities_Positive',
    'air conditioning_Positive', 'view_Positive', 'environment_Positive', 'security_Positive'
]


# Hàm tạo tensor từ DataFrame
def create_tensor_from_df(df):
    # Lấy danh sách trạng thái và khách sạn duy nhất
    statuses = sorted(df['state'].unique())
    hotels = sorted(df['id'].unique())
    
    num_statuses_actual = len(statuses)
    num_hotels_actual = len(hotels)
    
    # Ánh xạ trạng thái và khách sạn thành chỉ số
    status_to_idx = {s: i for i, s in enumerate(statuses)}
    hotel_to_idx = {h: i for i, h in enumerate(hotels)}
    
    # Tạo tensor với giá trị NaN
    tensor = np.full((num_statuses_actual, num_hotels_actual, num_criteria), np.nan)
    
    # Điền dữ liệu vào tensor
    for _, row in df.iterrows():
        status_idx = status_to_idx[row['state']]
        hotel_idx = hotel_to_idx[row['id']]
        for crit_idx, crit in enumerate(criteria):
            tensor[status_idx, hotel_idx, crit_idx] = row[crit]
    
    return tensor, status_to_idx, hotel_to_idx

# Hàm tiền xử lý: Điền giá trị thiếu bằng trung bình
def preprocess_tensor(tensor):
    # Điền NaN bằng trung bình của mỗi tiêu chí
    for k in range(tensor.shape[2]):
        slice_k = tensor[:, :, k]
        mean_k = np.nanmean(slice_k)
        slice_k[np.isnan(slice_k)] = mean_k
        tensor[:, :, k] = slice_k
    return tensor

# Hàm dự đoán điểm số cho (trạng thái, khách sạn)
def predict_scores(status_idx, hotel_idx, factors, weights):
    u, v, w = factors
    scores = np.zeros(num_criteria)
    for r in range(len(weights)):
        scores += weights[r] * u[status_idx, r] * v[hotel_idx, r] * w[:, r]
    # Đảm bảo điểm số nằm trong khoảng [0, 1]
    scores = np.clip(scores, 0, 1)
    return scores

# Hàm lưu model
def save_model(weights, factors, status_to_idx, hotel_to_idx, filename='model.pkl'):
    model_data = {
        'weights': weights,
        'factors': factors,
        'status_to_idx': status_to_idx,
        'hotel_to_idx': hotel_to_idx,
        'criteria': criteria
    }
    with open(filename, 'wb') as f:
        pickle.dump(model_data, f)
    print(f"Model đã được lưu vào {filename}")

# Hàm tải model
def load_model(filename='model.pkl'):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} không tồn tại")
    with open(filename, 'rb') as f:
        model_data = pickle.load(f)
    return (model_data['weights'], model_data['factors'],
            model_data['status_to_idx'], model_data['hotel_to_idx'], model_data['criteria'])

# Hàm dự đoán sử dụng model đã lưu
def predict_with_saved_model(status, hotel, model_filename='model.pkl'):
    weights, factors, status_to_idx, hotel_to_idx, criteria = load_model(model_filename)
    
    # Kiểm tra trạng thái và khách sạn có trong dữ liệu không
    if status not in status_to_idx:
        raise ValueError(f"Trạng thái '{status}' không có trong dữ liệu")
    if hotel not in hotel_to_idx:
        raise ValueError(f"Khách sạn '{hotel}' không có trong dữ liệu")
    
    status_idx = status_to_idx[status]
    hotel_idx = hotel_to_idx[hotel]
    predicted_scores = predict_scores(status_idx, hotel_idx, factors, weights)
    
    # In kết quả
    print(f"\nDự đoán điểm số cho trạng thái '{status}', khách sạn '{hotel}':")
    for i, (crit, score) in enumerate(zip(criteria, predicted_scores), 1):
        print(f"{crit}: {score:.3f}")
    
    return predicted_scores



In [34]:

print("Tạo tensor từ dữ liệu...")
tensor, status_to_idx, hotel_to_idx = create_tensor_from_df(df)

print("Tiền xử lý tensor...")
tensor = preprocess_tensor(tensor)

print("Phân rã tensor bằng CP Decomposition...")
weights, factors = parafac(tensor, rank=rank, init='random', tol=1e-6)

print("Lưu model...")
save_model(weights, factors, status_to_idx, hotel_to_idx, filename='CF_state_for_user.pkl')

# Dự đoán điểm số cho một ví dụ (sử dụng model vừa huấn luyện)
status = 'Cặp đôi'  # Trạng thái mẫu
hotel = df['id'].iloc[0]  # Lấy khách sạn mẫu
status_idx = status_to_idx[status]
hotel_idx = hotel_to_idx[hotel]
predicted_scores = predict_scores(status_idx, hotel_idx, factors, weights)

# In kết quả
print(f"\nDự đoán điểm số cho trạng thái '{status}', khách sạn '{hotel}' (từ model vừa huấn luyện):")
for i, (crit, score) in enumerate(zip(criteria, predicted_scores), 1):
    print(f"{crit}: {score:.3f}")

# Dự đoán lại bằng model đã lưu để kiểm tra
print("\nKiểm tra dự đoán với model đã lưu...")
predict_with_saved_model(status, hotel, model_filename='CF_state_for_user.pkl')



Tạo tensor từ dữ liệu...
Tiền xử lý tensor...
Phân rã tensor bằng CP Decomposition...
Lưu model...
Model đã được lưu vào CF_state_for_user.pkl

Dự đoán điểm số cho trạng thái 'Cặp đôi', khách sạn '10000593' (từ model vừa huấn luyện):
location_Positive: 0.782
room_Positive: 0.799
cleanliness_Positive: 0.837
comfort_Positive: 0.838
service_Positive: 0.927
staff_Positive: 0.965
wifi_Positive: 0.978
food_Positive: 0.831
bathroom_Positive: 0.760
parking_Positive: 0.990
value_Positive: 0.873
facilities_Positive: 0.805
air conditioning_Positive: 0.986
view_Positive: 0.986
environment_Positive: 0.832
security_Positive: 0.978

Kiểm tra dự đoán với model đã lưu...

Dự đoán điểm số cho trạng thái 'Cặp đôi', khách sạn '10000593':
location_Positive: 0.782
room_Positive: 0.799
cleanliness_Positive: 0.837
comfort_Positive: 0.838
service_Positive: 0.927
staff_Positive: 0.965
wifi_Positive: 0.978
food_Positive: 0.831
bathroom_Positive: 0.760
parking_Positive: 0.990
value_Positive: 0.873
facilities_Posi

array([0.78193132, 0.7993162 , 0.83660257, 0.83774752, 0.9265349 ,
       0.96520172, 0.97766524, 0.83083261, 0.76032452, 0.98985657,
       0.87293253, 0.80485626, 0.98626779, 0.98599781, 0.83204605,
       0.97834679])

In [35]:
import pandas as pd

# Giả sử result_groupby_id là dict chứa dữ liệu

rows = []

for id_, id_data in result_groupby_id.items():
    criteria_summary = id_data['summary']['criteria_summary']
    row = {'id': id_}
    
    for criterion, sentiment_scores in criteria_summary.items():
        positive_score = sentiment_scores.get('Positive', 0.0)
        row[f"{criterion}_Positive"] = positive_score
    
    rows.append(row)

# Tạo DataFrame
df = pd.DataFrame(rows)


In [41]:
df.head()

,id,service_Positive,staff_Positive,room_Positive,location_Positive,food_Positive,facilities_Positive,view_Positive,environment_Positive,cleanliness_Positive,bathroom_Positive,comfort_Positive,wifi_Positive,parking_Positive,value_Positive,air conditioning_Positive,security_Positive
0,10000593,0.925743,0.968944,0.784314,0.823232,0.825397,0.788732,0.965909,0.832558,0.858268,0.736842,0.807018,0.969072,0.96875,0.861635,0.970874,0.971831
1,10000695,0.500000,0.666667,0.500000,0.250000,0.333333,0.666667,0.500000,0.500000,0.666667,0.500000,0.000000,1.000000,1.00000,0.333333,1.000000,1.000000
2,10001679,0.785714,1.000000,0.666667,0.666667,0.833333,0.733333,0.909091,0.900000,1.000000,0.750000,0.666667,1.000000,1.00000,0.875000,1.000000,1.000000
3,10003086,0.857143,1.000000,0.736842,0.555556,0.600000,0.615385,0.750000,0.700000,0.857143,0.200000,0.687500,1.000000,1.00000,0.625000,1.000000,NaN
4,10003667,0.666667,1.000000,0.666667,1.000000,1.000000,0.500000,1.000000,0.800000,0.833333,0.428571,0.833333,1.000000,1.00000,0.500000,1.000000,1.000000


In [42]:
df.to_csv('review_criteria_summary_for_hotel.csv', index=False, encoding='utf-8-sig')


In [47]:
import CF_user_scores

In [49]:
CF = CF_user_scores.CF_user_scores()

state_model = CF.load_model('CF_state_for_user.pkl')
country_model = CF.load_model('CF_country_for_user.pkl')

df = pd.read_csv('review_criteria_summary_for_hotel.csv', encoding='utf-8-sig')



In [50]:
hotel_ids = ['10000593']
results = CF.aggregate_prediction(hotel_ids=hotel_ids,  
                        result_groupby_id=df,
                        state_model=state_model, 
                        country_model=country_model, print_result=True)



Dự đoán điểm số từ dữ liệu tổng hợp:

Khách sạn '10000593':
  location_Positive: 0.926
  room_Positive: 0.969
  cleanliness_Positive: 0.784
  comfort_Positive: 0.823
  service_Positive: 0.825
  staff_Positive: 0.789
  wifi_Positive: 0.966
  food_Positive: 0.833
  bathroom_Positive: 0.858
  parking_Positive: 0.737
  value_Positive: 0.807
  facilities_Positive: 0.969
  air conditioning_Positive: 0.969
  view_Positive: 0.862
  environment_Positive: 0.971
  security_Positive: 0.972


In [46]:
result = CF.aggregate_prediction(['10000593'], 'Ba Lan', 'Cặp đôi', state_model=state_model, 
                        country_model=country_model, print_result=True)


Dự đoán điểm số cho trạng thái 'Cặp đôi', quốc gia 'Ba Lan':

Khách sạn '10000593':
  location_Positive: 0.766
  room_Positive: 0.673
  cleanliness_Positive: 0.782
  comfort_Positive: 0.710
  service_Positive: 0.842
  staff_Positive: 0.933
  wifi_Positive: 0.956
  food_Positive: 0.753
  bathroom_Positive: 0.643
  parking_Positive: 0.970
  value_Positive: 0.821
  facilities_Positive: 0.684
  air conditioning_Positive: 0.965
  view_Positive: 0.898
  environment_Positive: 0.778
  security_Positive: 0.960


In [ ]:
with open('calculate_final_score_global_data.pkl', 'rb') as f:
    calculate_final_score_global_data = pickle.load(f)